In [4]:
import numpy as np 
import random
from scipy.optimize import minimize

# generate training and tesing data
Nt = 2
Nr = 4
# generate channel
H = np.sqrt(1/2)*(np.random.randn(Nr,Nt)+1j*np.random.randn(Nr,Nt))

In [7]:
# generate transmit signal
def generate_random_bit_sequence(length):
    return ''.join(random.choice('01') for _ in range(length))

def qam16_modulation(binary_input):
    mapping = {
        '0000': (-3-3j),
        '0001': (-3-1j),
        '0010': (-3+3j),
        '0011': (-3+1j),
        '0100': (-1-3j),
        '0101': (-1-1j),
        '0110': (-1+3j),
        '0111': (-1+1j),
        '1000': (3-3j),
        '1001': (3-1j),
        '1010': (3+3j),
        '1011': (3+1j),
        '1100': (1-3j),
        '1101': (1-1j),
        '1110': (1+3j),
        '1111': (1+1j)
    }
    return mapping.get(binary_input, "Invalid binary input")/np.sqrt(10)

def generate_x_sequence(length, Nt):
    total_bits_sequence = generate_random_bit_sequence(length*Nt*4)
    bits_sequence = [total_bits_sequence[i:i+4] for i in range(0, len(total_bits_sequence), 4)]
    x_sequence = [np.array([qam16_modulation(bits_sequence[i]), qam16_modulation(bits_sequence[i+1])]) for i in range(0, len(bits_sequence), Nt)]
    return bits_sequence, x_sequence

# noise
SNR_dB = 50
SNR = 10.0**(SNR_dB/10.0)
def generate_noise(SNR, Nr):
    return np.sqrt(1/(2*SNR))*(np.random.randn(Nr,1)+1j*np.random.randn(Nr,1))

# generate training and tesing data
def generate_data(Nr,Nt,SNR,length,H_channel):
    bits_sequence, x_sequence = generate_x_sequence(length, Nt)
    n_sequence = [generate_noise(SNR, Nr) for i in range(length)]
    y_sequence = [np.dot(H_channel, x_sequence[i].reshape(Nt,1)) + n_sequence[i] for i in range(length)]
    return bits_sequence, x_sequence, y_sequence

training_length = 1000
bits_sequence, x_sequence, y_sequence = generate_data(Nr,Nt,SNR,training_length,H)

array([[-0.00174988+2.97692067e-03j],
       [-0.00090566-6.25427341e-03j],
       [ 0.00072461-1.55710080e-03j],
       [-0.00102848+4.90405903e-05j]])

In [6]:
def bits2signals(bits):
    # bits: input binary string with length of (4*Nt) 
    return np.array([qam16_modulation(bits[i:i+4]) for i in range(0, len(bits), 4)]).reshape(Nt,1)
def calculate_layer1(H_hat, y):
    dimension_layer1 = 2**(4*Nt)
    output = {}
    for index in range(dimension_layer1):
        bits = str(bin(index)[2:].zfill(4*Nt))
        s = bits2signals(bits)
        error = y - np.dot(H_hat,s)
        value =  np.exp(-np.square(np.linalg.norm(error)))
        if value > 1e-5:
            output[bits] = value
    return output

def calculate_layer2(layer1_output):
    sum_exp = [[0 for i in range(2)] for j in range(4*Nt)]
    for bits in layer1_output:
        value = layer1_output[bits]
        for index in range(4*Nt):
            sum_exp[index][eval(bits[index])] += value
    output = {}
    for index in range(4*Nt):
        # llr = np.log(sum_exp[index][1]/sum_exp[index][0])
        output[index] = (sum_exp[index][1]+1e-10)/(sum_exp[index][1]+sum_exp[index][0]+1e-10)
    return output

def calculate_cross_entropy(layer2_output, true_sequence):
    dimension = len(true_sequence)
    entropy = 0
    for index in range(dimension):
        if true_sequence[index] == '1':
            entropy += (-np.log(layer2_output[index]))
    return entropy

def calculate_cost_function(H_hat_vec):
    H_hat = H_hat_vec[0:Nr*Nt].reshape(Nr,Nt)+1j*H_hat_vec[Nr*Nt:2*Nr*Nt].reshape(Nr,Nt)
    total_loss = 0
    for ii in range(training_length):
        layer1_output = calculate_layer1(H_hat, y_sequence[ii])
        layer2_output = calculate_layer2(layer1_output)
        true_sequence = ''.join(bits_sequence[ii*Nt+jj] for jj in range(Nt))
        total_loss += calculate_cross_entropy(layer2_output,true_sequence)
    mean_loss = total_loss/training_length
    print(mean_loss)
    return mean_loss
        


calculate_cost_function(np.hstack((np.real(H).reshape(8), np.imag(H).reshape(8))))

# out = minimize(calculate_cost_function, x0=np.sqrt(1/2)*(np.random.randn(Nr*Nt*2)), method="COBYLA", options={'maxiter':50,'catol':1e-6})
# print(out)

1.102552490183149


1.102552490183149

In [44]:

def detection(y, H_trained):
    layer1_output = calculate_layer1(H_trained, y)
    layer2_output = calculate_layer2(layer1_output)
    detect_result = ''
    for ii in range(len(layer2_output)):
        if(layer2_output[ii]>0.5):
            detect_result += '1'
        else:
            detect_result += '0'
    return(detect_result)

def count_differences(str1, str2):
    return sum(a != b for a, b in zip(str1, str2))

def calculate_BER(H_trained):
    # tesing set
    testing_length = 1000
    bits_sequence_testing, x_sequence_testing, y_sequence_testing = generate_data(Nr,Nt,SNR,testing_length,H)
    error = 0
    for ii in range(len(y_sequence_testing)):
        detect_result = detection(y_sequence_testing[ii], H_trained)
        true_sequence = ''.join(bits_sequence_testing[ii*Nt+jj] for jj in range(Nt))
        error += count_differences(detect_result, true_sequence)
    BER = error/(len(y_sequence_testing)*len(detect_result))
    return BER
# calculate_BER(H)


In [45]:
for iternum in [50, 100, 200, 400]:
    out = minimize(calculate_cost_function, x0=np.sqrt(1/2)*(np.random.randn(Nr*Nt*2)), method="COBYLA", options={'maxiter':iternum,'catol':1e-6})
    H_trained = out.x[0:Nr*Nt].reshape(Nr,Nt)+1j*out.x[Nr*Nt:2*Nr*Nt].reshape(Nr,Nt)
    BER = calculate_BER(H_trained)
    print(iternum, BER)

2.2663161390723867
2.2861237641991727
3.2394280985764015
2.17393721783385
1.937486319461204
1.9171805297890911
2.2530971899682224
1.6414138199238775
2.0445939313769066
1.8788045996180849
1.6427667841924452
1.792913360872592
1.798599130466499
2.00786808411242
2.36144914059184
1.5162306997853516
1.4653894024256338
1.2465010126980989
0.9988092934539151
0.921403604128174
0.9840022229068174
1.185295826047954
0.8400861278235969
0.7948864648231995
0.7962220921391423
0.6641633757018044
0.5722900215879704
0.5731532981298246
0.5344730389223494
0.5151478952114338
0.5757703728190265
0.5119585514554005
0.5973963930090423
0.4997547694513968
0.5705481635382417
0.533459166860056
0.4537793745749618
0.4464348575959349
0.4774766469308092
0.47465331229436636
0.4870297551462046
0.567959603273694
0.41549951927766055
0.47531024915975145
0.4104384377316019
0.43232852426574037
0.41339374923106453
0.3806415665717186
0.39799703768277717
0.3343885490015021
50 0.278125
3.378988873625121
2.8904381354411517
2.494267

KeyboardInterrupt: 

In [27]:
H

array([[ 0.33180004-0.06859886j, -0.58483981-0.18134615j],
       [-0.23536349-0.29148581j, -0.12561431-0.80242992j],
       [ 0.19487322+0.02846216j,  0.05014177-1.4547925j ],
       [ 0.47327803+0.39885556j, -1.02826642-0.24389847j]])